In [2]:
import gurobipy as gp

In [3]:
from gurobipy import Model, GRB, quicksum

In [4]:
# Conjuntos
I = list(range(12))  # Piezas, las primeras seis piezas son de tipo uno (2x4x1) y las ultimas seis piezas son de tipo 2 (2x3x2)
R = list(range(6))   # Rotaciones (6 posibles)

In [5]:
# Dimensiones de la caja 
W, L, H = 5,5,5

In [6]:
# Dimensiones de una pieza(ancho, largo, alto)
#Tipo 1
w1, l1, h1 = 2, 4, 1
#Tipo 2
w2,l2,h2= 2,3,2
# Definición de las rotaciones
rotaciones = [
    lambda w, l, h: (w, l, h),
    lambda w, l, h: (w, h, l),
    lambda w, l, h: (l, w, h),
    lambda w, l, h: (l, h, w),
    lambda w, l, h: (h, w, l),
    lambda w, l, h: (h, l, w)
]

# Diccionario  donde se especifica la dimension (ancho, largo, alto) de la pieza segun su tipo y rotación 
pieza_rotaciones = {
    1: {r: rotaciones[r](w1, l1, h1) for r in range(6)}, 2:{r: rotaciones[r](w2, l2, h2) for r in range(6)}
}

print(pieza_rotaciones)

{1: {0: (2, 4, 1), 1: (2, 1, 4), 2: (4, 2, 1), 3: (4, 1, 2), 4: (1, 2, 4), 5: (1, 4, 2)}, 2: {0: (2, 3, 2), 1: (2, 2, 3), 2: (3, 2, 2), 3: (3, 2, 2), 4: (2, 2, 3), 5: (2, 3, 2)}}


In [7]:
#Diccionario donde se especifica el ancho de la pieza segun tipo y rotación
w_ir = {piece: {rotation: pieza_rotaciones[piece][rotation][0] for rotation in pieza_rotaciones[piece]} for piece in pieza_rotaciones}
print(w_ir)

{1: {0: 2, 1: 2, 2: 4, 3: 4, 4: 1, 5: 1}, 2: {0: 2, 1: 2, 2: 3, 3: 3, 4: 2, 5: 2}}


In [8]:
#Diccionario donde se especifica el largo de la pieza segun tipo y rotación
l_ir = {piece: {rotation: pieza_rotaciones[piece][rotation][1] for rotation in pieza_rotaciones[piece]} for piece in pieza_rotaciones}
print(l_ir)

{1: {0: 4, 1: 1, 2: 2, 3: 1, 4: 2, 5: 4}, 2: {0: 3, 1: 2, 2: 2, 3: 2, 4: 2, 5: 3}}


In [9]:
#Diccionario donde se especifica el alto de la pieza segun tipo y rotación
h_ir = {piece: {rotation: pieza_rotaciones[piece][rotation][2] for rotation in pieza_rotaciones[piece]} for piece in pieza_rotaciones}
print(h_ir)

{1: {0: 1, 1: 4, 2: 1, 3: 2, 4: 4, 5: 2}, 2: {0: 2, 1: 3, 2: 2, 3: 2, 4: 3, 5: 2}}


In [10]:
# Constante M suficientemente grande
M = 100

In [11]:
# Crear el modelo
model = Model("3D-SKP")

Set parameter Username
Set parameter LicenseID to value 2616364
Academic license - for non-commercial use only - expires 2026-01-29


In [12]:
# Variables de decisión
X = model.addVars(I, R, vtype=GRB.BINARY, name="X")  # Indica si la pieza i se en cuentra en la caja y en cual rotación r
x = model.addVars(I, vtype=GRB.CONTINUOUS, name="x")  # Coordenada x de la pieza i 
y = model.addVars(I, vtype=GRB.CONTINUOUS, name="y")  # Coordenada y de la pieza i
z = model.addVars(I, vtype=GRB.CONTINUOUS, name="z")  # Coordenada z de la pieza i 
o=  model.addVars(I, vtype=GRB.BINARY, name="o") # Indica si la pieza i se encuentra en la caja 

In [13]:
# Variables auxiliares para separación
a = model.addVars(I, I, vtype=GRB.BINARY, name="a")  # Indica si la pieza j esta a la derecha de la pieza i 
b = model.addVars(I, I, vtype=GRB.BINARY, name="b")  # Indica si la pieza j esta a la izquierda de la pieza i
c = model.addVars(I, I, vtype=GRB.BINARY, name="c")  # Indica si la pieza j esta arriba de la pieza i
d = model.addVars(I, I, vtype=GRB.BINARY, name="d")  # Indica si la pieza j esta debajo de la pieza i
e = model.addVars(I, I, vtype=GRB.BINARY, name="e")  # Indica si la pieza j esta adelante de la pieza i
f = model.addVars(I, I, vtype=GRB.BINARY, name="f")  # Indica si la pieza j esta detras de la pieza i

In [14]:
# Función objetivo: maximizar el número de piezas colocadas
model.setObjective(quicksum(X[i, r] for i in I for r in R), GRB.MAXIMIZE)

In [15]:
#Si la pieza i no esta ubicada dentro de la caja no puede tener activa ninguna de las rotaciones r 
for i in I:
    for r in R:
        model.addConstr(X[i,r] <= o[i])

In [16]:
# Restricciones de capacidad de la caja teniendo en cuenta el tipo de pieza 
for i in I:
    if i >=0 and i<=5:
           model.addConstr(x[i] + quicksum(w_ir[1][r] * X[i, r] for r in R) <= W)
           model.addConstr(y[i] + quicksum(l_ir[1][r] * X[i, r] for r in R) <= L)
           model.addConstr(z[i] + quicksum(h_ir[1][r] * X[i, r] for r in R)<= H)
    else:
           model.addConstr(x[i] + quicksum(w_ir[2][r] * X[i, r] for r in R) <= W)
           model.addConstr(y[i] + quicksum(l_ir[2][r] * X[i, r] for r in R)<= L)
           model.addConstr(z[i] + quicksum(h_ir[2][r] * X[i, r] for r in R)<= H)



In [17]:
# Para cada par de piezas ij, debe haber una relación directa entre ambas en uno de sus ejes 
for i in I:
    for j in I:
        if i<j:
           model.addConstr(a[i,j]+b[i,j]+c[i,j]+d[i,j]+e[i,j]+f[i,j]== 1)
    

In [18]:
# Una pieza solo puede tener una rotación activa
for i in I:
    model.addConstr(quicksum(X[i, r] for r in R) <= 1, f"OneRotation_{i}")

In [19]:
# Restricciones de no superposición segun el tipo de pieza ij 
for i in I:
   for j in I:
       if i >=0 and i<=5 and i != j:
         model.addConstr(x[i] + quicksum(w_ir[1][r]* X[i,r] for r in R)<= x[j] + M * (1 - a[i, j]))
         model.addConstr(y[i] + quicksum(l_ir[1][r]* X[i,r] for r in R) <= y[j] + M * (1- f[i, j]))
         model.addConstr(z[i] + quicksum(h_ir[1][r]* X[i,r] for r in R)<= z[j] + M * (1 - c[i, j]))
       if j>=0 and j<=5 and i!=j :
         model.addConstr(x[j] + quicksum(w_ir[1][r]* X[j, r]for r in R)<= x[i] + M * (1 - b[i,j]))
         model.addConstr(y[j] + quicksum(l_ir[1][r]* X[j, r] for r in R)<= y[i] + M * (1- e[i,j]))
         model.addConstr(z[j] + quicksum(h_ir[1][r]* X[j, r] for r in R)<= z[i] + M * (1- d[i,j]))
       if j >=6 and i != j:
         model.addConstr(x[j] + quicksum(w_ir[2][r] * X[j, r] for r in R)<= x[i] + M * (1 - b[i,j]))
         model.addConstr(y[j] + quicksum(l_ir[2][r] * X[j, r] for r in R) <= y[i] + M * (1 - e[i,j]))
         model.addConstr(z[j] + quicksum(h_ir[2][r] * X[j, r] for r in R)<= z[i] + M * (1- d[i,j]))     
       if i >=6 and i != j:
         model.addConstr(x[i] + quicksum(w_ir[2][r] * X[i, r] for r in R)<= x[j] + M * (1 - a[i, j]))
         model.addConstr(y[i] + quicksum(l_ir[2][r] * X[i, r] for r in R) <= y[j] + M * (1 - f[i, j]))
         model.addConstr(z[i] + quicksum(h_ir[2][r] * X[i, r] for r in R)<= z[j] + M * (1 - c[i, j]))
 




In [20]:
# Resolver el modelo
model.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 978 rows, 984 columns and 7992 nonzeros
Model fingerprint: 0x90c0c42c
Variable types: 36 continuous, 948 integer (948 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 468 rows and 498 columns
Presolve time: 0.02s
Presolved: 510 rows, 486 columns, 3618 nonzeros
Variable types: 36 continuous, 450 integer (450 binary)

Root relaxation: objective 1.200000e+01, 188 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [21]:
# Mostrar resultados
if model.status == GRB.OPTIMAL:
    print("\nSolución óptima encontrada:\n")
    for i in I:
        for r in R:
            if X[i, r].x > 0.5:
                print(f"Pieza {i} colocada con rotación {r} en ({x[i].x:.2f}, {y[i].x:.2f}, {z[i].x:.2f})")
else:
    print("No se encontró solución óptima.")



Solución óptima encontrada:

Pieza 0 colocada con rotación 1 en (0.00, 4.00, 1.00)
Pieza 1 colocada con rotación 5 en (4.00, 1.00, 3.00)
Pieza 2 colocada con rotación 2 en (0.00, 0.00, 4.00)
Pieza 3 colocada con rotación 2 en (1.00, 3.00, 0.00)
Pieza 4 colocada con rotación 1 en (3.00, 0.00, 0.00)
Pieza 5 colocada con rotación 5 en (0.00, 0.00, 0.00)
Pieza 6 colocada con rotación 3 en (0.00, 0.00, 2.00)
Pieza 7 colocada con rotación 5 en (2.00, 2.00, 3.00)
Pieza 8 colocada con rotación 4 en (3.00, 1.00, 0.00)
Pieza 9 colocada con rotación 3 en (2.00, 3.00, 1.00)
Pieza 10 colocada con rotación 0 en (1.00, 0.00, 0.00)
Pieza 11 colocada con rotación 1 en (0.00, 2.00, 2.00)
